# Constants and Configuration

### Content

1. [Constants](#Const)
    1. [Hard Coded](#ConstHard)
    2. [Conigurable](#ConstConf)
    3. [Where to put constants?](#ConstWhere)
2. [Configuration](#Conf)
    1. [Config files](#ConfFiles)
    2. [Accessing configuration values](#ConfAccess)
    3. [Default Configuration](#ConfDefault)


##  1. <a id='Const'> Constants </a>
Constants are values that, once initialized, are never changed during the runtime of a program. In Python constants are assigned to variables with capital letters by convention, and vice versa, variables with capital letters are supposed to be constants.

In principle there are about four ways to define a constant's value:
- _hard coding_: the value is defined in the python code directly
- _argument_: the value is taken from an execution argument
- _context_: the value is derived from the environmental context of the exection, e.g., the current working directory or the date-time of execution start.
- _configuation_: read from a file or database

In CLIMADA, we only use _hard coding_ and _configuration_ to assign values to constants.

###  1.A. <a id='ConstHard'> Hard Coded </a>
Hard coding constants is the prefered way to deal with strings that are used in several locations, e.g. to identify objects or files.

In [15]:
# suboptimal
my_dict = {'x': 4}
if my_dict['x'] > 3:
    print('well, arh, ...')

well, arh, ...


In [14]:
# good
X = 'x'
my_dict = {X: 4}
if my_dict[X] > 3:
    print('yeah!')

yeah!


In [17]:
# possibly overdoing it
X = 'x'
Y = "this doesn't mean that every string must be a constant"
my_dict = {X: 4}
if my_dict[X] > 3:
    print(Y)

this doesn't mean that every string must be a constant


In [25]:
import pandas as pd
X = 'x'
df = pd.DataFrame({'x':[1,2,3], 'y':[4,5,6]})
try:
    df.X
except AttributeError:
    print("this wouldn't work")
    df[X]
    print("this would but it's less pretty")
try:
    df.x
    print("sometimes constants are pretty useless")
except:
    pass

this wouldn't work
this would but it's less pretty
sometimes constants are pretty useless


###  1.B. <a id='ConstConf'> Configurable </a>
When it comes to absolute pathes, it is urgently suggested to not use hard coded constant values, for the obvious reasons. But also relative pathes can cause problemns. In particular, they may point to a location where the user has not sufficient access permissions. In order to avoid these problems, _all_ pathes constants in CLIMADA are supposed to be defined through configuration.

The same applies tu urls to external resources, databases or websites. Since they may change at any time, there addresses are supposed to be defined through configuration. Like this it will be possible to access them without the need of tampering with the source code or waiting for a new release.

Another category of constants that should go into the configuration file are system specifications, such as number of CPU's available for CLIMADA or memory settings.

###  1.C. <a id='ConstWhere'> Where to put constants? </a>
As a general rule constants are defined in the module where they intrinsically belong to. If they belong equally to different modules though or they are meant to be used globally, there is the module `climada.util.constants` which is compiling constants CLIMADA wide.

##  2. <a id='Conf'> Configuration </a>
In CLIMADA all constants that are not hard coded are defined in configuration files. These are, generally spoken, all constants that a user may want (or need) to change without changing the CLIMADA installation.\
We have tried to make the configuration of CLIMADA flexible, transparent, easy to define and easy to use.

###  2.A. <a id='ConfFiles'> Configuration Files </a>
Configuration files are files in _json_ format with the name `climada.conf` in json format. One can have several of them, complementing one another. CLIMADA looks for configuration files upon `import climada`.

The first place to look for a configuration file is `climada/conf/` within the installation directory, then `~/climada/` is checked and at last the current working directory. At each of the three locations, the path is followed upwards until the root of the path is reached. When ever a `climada.conf` file is found it is taken into account. When two config files are defining the same value, the priorities are:\
`./climada.conf` \
\> `../climada.conf` \
\> `../../climada.conf` \
\> `~/climada/climada.conf` \
\> `~/climada.conf` \
\> `installation_dir/climada/conf/climada.conf`

In [12]:
from climada import CONFIG

In [10]:
CONFIG.__dict__

{'_root': {_comment: this is the climada default configuration, it will be (partially) superseded by the first climada.conf files that are present in the user's home directory and the working directory or its parent directories, local_data: {system: ~/climada/data, demo: ~/climada/demo/data, save_dir: ./results}, exposures: {crop_production: {local_data: ~/climada/data/ISIMIP_crop}, litpop: {resources: {world_bank_inc_group: http://databank.worldbank.org/data/download/site-content/OGHIST.xls}}, spam_agrar: {resources: {spam2005_api_access: https://dataverse.harvard.edu/api/access/datafile/:persistentId?persistentId=doi:10.7910/DVN/DHXBJX/, spam2005_dataset: https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/DHXBJX}}}, hazard: {drought: {resources: {spei_file_url: http://digital.csic.es/bitstream/10261/153475/8}}, landslide: {resources: {opensearch: https://pmmpublisher.pps.eosdis.nasa.gov/opensearch, climatology_monthly: https://svs.gsfc.nasa.gov/vis/a000000/a0046

In [9]:
CONFIG.local_data.system.__class__

climada.util.config.Config